In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

In [2]:
SaveFileName = "test_fea.xlsx"

In [3]:
test_base = pd.read_csv('test/test_base.csv')
# train_label = pd.read_csv('train/train_label.csv')
test_op = pd.read_csv('test/test_op.csv')
test_trans = pd.read_csv('test/test_trans.csv')


In [58]:
test_fea = pd.read_excel('test_fea.xlsx')

### 提取天数信息

In [4]:
# 将tim_ff列提出天数并考察密度分布，
# 划分出时间切片1 7 15天做特征计算和统计
def extract_day(s):
    char_list = s.split(' ')
    day = eval(char_list[0])
    return day


In [5]:
test_op['day'] = test_op['tm_diff'].apply(lambda s:extract_day(s))

In [6]:
test_trans['day'] = test_trans['tm_diff'].apply(lambda s:extract_day(s))

In [7]:
user = test_base['user'].to_list()

In [8]:
def CountOpt(s,optData,noOptUser):
    if s not in noOptUser:
        return optData[s]
    else:
        return 0

In [9]:
test_fea = test_base

对操作次数进行统计

In [12]:
def TotalOpt(df,day,attr):
    limitDay =df[ df['day'] <= day]
    optCount = limitDay.groupby(['user']).size()
    haveRecordUser = optCount.index.to_list()
    noOptUser = list(filter(lambda x: x not in haveRecordUser,user))
    optData = optCount.to_dict()
    test_fea[attr] = test_fea.apply(lambda s:CountOpt(s['user'],optData,noOptUser),axis=1)
    

In [13]:
attr_Opt = ['p_oneDayOpt','p_7DayOpt','p_15DayOpt']
day = [1,7,15,30]

TotalOpt(test_op,day[0],attr_Opt[0])
TotalOpt(test_op,day[1],attr_Opt[1])
TotalOpt(test_op,day[2],attr_Opt[2])

对ip类型进行统计

In [14]:
def CountIpType(df,day,attr):
    limitDay =df[ df['day'] <= day]
    CountIp = limitDay.groupby(['user']).apply(lambda s: len(list(s.groupby(['ip']).groups.keys())))
    haveRecordUser = CountIp.index.to_list()
    noOptUser = list(filter(lambda x: x not in haveRecordUser,user))
    optData = CountIp.to_dict()

    test_fea[attr] = test_fea.apply(lambda s:CountOpt(s['user'],optData,noOptUser),axis=1)


In [15]:
CountIpType(test_op,1,'p_oneDayOptIpType')
CountIpType(test_op,7,'p_7DayOptIpType')
CountIpType(test_op,15,'p_15DayOptIpType')


对操作渠道进行统计

In [16]:
# 对渠道进行统计时间分片统计 

def CountChannelType(df,day,attr):
    limitDay =df[ df['day'] <= day]
    Count = limitDay.groupby(['user']).apply(lambda s: len(list(s.groupby(['channel']).groups.keys())))
    haveRecordUser = Count.index.to_list()
    noOptUser = list(filter(lambda x: x not in haveRecordUser,user))
    optData = Count.to_dict()

    test_fea[attr] = test_fea.apply(lambda s:CountOpt(s['user'],optData,noOptUser),axis=1)



In [17]:
CountChannelType(test_op,1,'p_oneDayOptChanType')
CountChannelType(test_op,7,'p_7DayOptChanType')
CountChannelType(test_op,15,'p_15DayOptChanType')


## 对交易信息进行统计

In [22]:
def TotalMoney(df,day,attr):
    limitDay =df[ df['day'] <= day]
    optCount = limitDay.groupby(['user']).apply(lambda s:s['amount'].sum())
    haveRecordUser = optCount.index.to_list()
    noOptUser = list(filter(lambda x: x not in haveRecordUser,user))

    optData = optCount.to_dict()

    # test_fea[attr] = test_fea.apply(lambda s:[optData[s] if s['user'] not in noOptUser else 0],axis=1)
    test_fea[attr] = test_fea.apply(lambda s:CountOpt(s['user'],optData,noOptUser),axis=1)
  

In [23]:
TotalMoney(test_trans,1,'AllMon_1') 
TotalMoney(test_trans,7,'AllMon_7') 
TotalMoney(test_trans,15,'AllMon_15') 
TotalMoney(test_trans,30,'AllMon_30') 

In [24]:
test_fea.to_excel('test_fea.xlsx',index=False)

交易金额的统计（累计、均值、最大、最小、方差）

In [10]:
limitOneDay = test_trans[test_trans['day']<=1]
limit30Day = test_trans[test_trans['day']<=30]

In [11]:
limit7Day = test_trans[test_trans['day']<=7]
limit15Day = test_trans[test_trans['day']<=15]

### 1天金额统计

In [27]:
# 1天金额均值、最大、最小值、方差
# 还要对交易次数进行统计 --
oneMax = limitOneDay.groupby(['user']).apply(lambda s:s['amount'].max())
oneMin = limitOneDay.groupby(['user']).apply(lambda s:s['amount'].min())
oneMean = limitOneDay.groupby(['user']).apply(lambda s:s['amount'].mean())
oneStd = limitOneDay.groupby(['user']).apply(lambda s:s['amount'].std())

In [28]:
_1DayinUser = oneMax.index.to_list()

In [29]:
_1DayNotInUser = list(filter(lambda x: x not in _1DayinUser,user))

In [30]:
test_fea['Mean'+'过去1天累计交易金额'] = test_fea.apply(lambda s:CountOpt(s['user'],oneMean.to_dict(),_1DayNotInUser),axis=1) 
test_fea['Std'+'过去1天累计交易金额'] = test_fea.apply(lambda s:CountOpt(s['user'],oneStd.to_dict(),_1DayNotInUser),axis=1)  
test_fea['Max'+'过去1天累计交易金额'] = test_fea.apply(lambda s:CountOpt(s['user'],oneMax.to_dict(),_1DayNotInUser),axis=1) 
test_fea['Min'+'过去1天累计交易金额'] =test_fea.apply(lambda s:CountOpt(s['user'],oneMin.to_dict(),_1DayNotInUser),axis=1)  

### 30天金额统计

In [31]:
# 1天金额均值、最大、最小值、方差
# 还要对交易次数进行统计 --
_30Max = limit30Day.groupby(['user']).apply(lambda s:s['amount'].max())
_30Min = limit30Day.groupby(['user']).apply(lambda s:s['amount'].min())
_30Mean = limit30Day.groupby(['user']).apply(lambda s:s['amount'].mean())
_30Std = limit30Day.groupby(['user']).apply(lambda s:s['amount'].std())

In [32]:
# user = list(train_base['user'])
_30DayinUser = _30Max.index.to_list()
_30DayNotInUser = list(filter(lambda x: x not in _30DayinUser,user))

In [34]:
test_fea['Mean'+'过去30天累计交易金额'] = test_fea.apply(lambda s:CountOpt(s['user'],_30Mean.to_dict(),_30DayNotInUser),axis=1) 
test_fea['Std'+'过去30天累计交易金额'] = test_fea.apply(lambda s:CountOpt(s['user'],_30Std.to_dict(),_30DayNotInUser),axis=1)  
test_fea['Max'+'过去30天累计交易金额'] = test_fea.apply(lambda s:CountOpt(s['user'],_30Max.to_dict(),_30DayNotInUser),axis=1) 
test_fea['Min'+'过去30天累计交易金额'] =test_fea.apply(lambda s:CountOpt(s['user'],_30Min.to_dict(),_30DayNotInUser),axis=1)  

### 15天金额统计

In [35]:
# 1天金额均值、最大、最小值、方差
# 还要对交易次数进行统计 --
_15Max = limit15Day.groupby(['user']).apply(lambda s:s['amount'].max())
_15Min = limit15Day.groupby(['user']).apply(lambda s:s['amount'].min())
_15Mean = limit15Day.groupby(['user']).apply(lambda s:s['amount'].mean())
_15Std = limit15Day.groupby(['user']).apply(lambda s:s['amount'].std())

In [36]:
# user = list(train_base['user'])
_15DayinUser = _15Max.index.to_list()
_15DayNotInUser = list(filter(lambda x: x not in _15DayinUser,user))

In [37]:
test_fea['Mean'+'过去15天累计交易金额'] = test_fea.apply(lambda s:CountOpt(s['user'],_15Mean.to_dict(),_15DayNotInUser),axis=1) 
test_fea['Std'+'过去15天累计交易金额'] = test_fea.apply(lambda s:CountOpt(s['user'],_15Std.to_dict(),_15DayNotInUser),axis=1)  
test_fea['Max'+'过去15天累计交易金额'] = test_fea.apply(lambda s:CountOpt(s['user'],_15Max.to_dict(),_15DayNotInUser),axis=1) 
test_fea['Min'+'过去15天累计交易金额'] =test_fea.apply(lambda s:CountOpt(s['user'],_15Min.to_dict(),_15DayNotInUser),axis=1)  

### 7天金额统计

In [38]:
# 1天金额均值、最大、最小值、方差
# 还要对交易次数进行统计 --
_7Max = limit7Day.groupby(['user']).apply(lambda s:s['amount'].max())
_7Min = limit7Day.groupby(['user']).apply(lambda s:s['amount'].min())
_7Mean = limit7Day.groupby(['user']).apply(lambda s:s['amount'].mean())
_7Std = limit7Day.groupby(['user']).apply(lambda s:s['amount'].std())

In [39]:
# user = list(train_base['user'])
_7DayinUser = _7Max.index.to_list()
_7DayNotInUser = list(filter(lambda x: x not in _7DayinUser,user))

In [40]:
test_fea['Mean'+'过去7天累计交易金额'] = test_fea.apply(lambda s:CountOpt(s['user'],_7Mean.to_dict(),_7DayNotInUser),axis=1) 
test_fea['Std'+'过去7天累计交易金额'] = test_fea.apply(lambda s:CountOpt(s['user'],_7Std.to_dict(),_7DayNotInUser),axis=1)  
test_fea['Max'+'过去7天累计交易金额'] = test_fea.apply(lambda s:CountOpt(s['user'],_7Max.to_dict(),_7DayNotInUser),axis=1) 
test_fea['Min'+'过去7天累计交易金额'] =test_fea.apply(lambda s:CountOpt(s['user'],_7Min.to_dict(),_7DayNotInUser),axis=1)  

交易次数的统计（累计和均值）

In [12]:
# 累计交易次数
_1times = limitOneDay.groupby(['user']).size()
_7times = limit7Day.groupby(['user']).size()
_15times = limit15Day.groupby(['user']).size()
_30times = limit30Day.groupby(['user']).size()

In [44]:
# 累计交易次数统计
test_fea['过去1天累计交易次数'] = test_fea.apply(lambda s:CountOpt(s['user'],_1times.to_dict(),_1DayNotInUser),axis=1)  
test_fea['过去7天累计交易次数'] = test_fea.apply(lambda s:CountOpt(s['user'],_7times.to_dict(),_7DayNotInUser),axis=1)  
test_fea['过去15天累计交易次数'] = test_fea.apply(lambda s:CountOpt(s['user'],_15times.to_dict(),_15DayNotInUser),axis=1)  
test_fea['过去30天累计交易次数'] = test_fea.apply(lambda s:CountOpt(s['user'],_30times.to_dict(),_30DayNotInUser),axis=1)  

In [14]:
_7timesM = _7times/7
_15timesM = _15times/15
_30timesM = _30times/30

In [46]:
# 平均交易次数统计
test_fea['过去7天平均交易次数'] = test_fea.apply(lambda s:CountOpt(s['user'],_7timesM.to_dict(),_7DayNotInUser),axis=1)  
test_fea['过去15天平均交易次数'] = test_fea.apply(lambda s:CountOpt(s['user'],_15timesM.to_dict(),_15DayNotInUser),axis=1)  
test_fea['过去30天平均交易次数'] = test_fea.apply(lambda s:CountOpt(s['user'],_30timesM.to_dict(),_30DayNotInUser),axis=1)  

交易平台类型计数统计

In [51]:
CountPlat = limit30Day.groupby(['user']).apply(lambda s: len(list(s.groupby(['platform']).groups.keys())))

test_fea['过去30天交易平台类型'] = test_fea.apply(lambda s:CountOpt(s['user'],CountPlat.to_dict(),_30DayNotInUser),axis=1)  

In [76]:
def Count1wan(x):
    count = 0
    for item in x:
        if item[4] > 10000:
            count+=1
    return count


In [77]:
# 过去1天大于1万交易的次数

_1over = limitOneDay.groupby(['user']).apply(lambda s:Count1wan(s.values))

In [79]:
test_fea['过去1天交易超过1万元次数'] = test_fea.apply(lambda s:CountOpt(s['user'],_1over.to_dict(),_1DayNotInUser),axis=1)  

In [82]:
FileName = "new_test_fea.csv"

In [81]:
test_fea.to_csv(FileName,index=False)

In [83]:
test_fea.head(3)

,user,sex,age,provider,level,verified,using_time,regist_type,card_a_cnt,card_b_cnt,...,Min过去7天累计交易金额,过去1天累计交易次数,过去7天累计交易次数,过去15天累计交易次数,过去30天累计交易次数,过去7天平均交易次数,过去15天平均交易次数,过去30天平均交易次数,过去30天交易平台类型,过去1天交易超过1万元次数
0,Test_02230,category 0,24914,category 0,category 2,category 0,24735,category 7,24743,24725,...,25315.0,0,16,37,104,2.285714,2.466667,3.466667,3,0
1,Test_21691,category 0,24920,category 0,category 2,category 0,24725,category 2,24725,24719,...,48351.0,0,4,4,4,0.571429,0.266667,0.133333,2,0
2,Test_31857,category 1,24877,category 2,category 2,category 0,24722,category 3,24725,24712,...,0.0,0,0,4,8,0.000000,0.266667,0.266667,3,0


In [56]:
# 特征交互？type1计算?

limOneDayIn = limitOneDay[(limitOneDay['tunnel_in'].notna())]
limOneDayOut = limitOneDay[(limitOneDay['tunnel_out'].notna())]


In [ ]:
def CountType1(df,day,attr):
    
    limitDay =df[ df['day'] <= day]
    CountIp = limitDay.groupby(['user']).apply(lambda s: len(list(s.groupby(['type1']).groups.keys())))

    haveRecordUser = CountIp.index.to_list()
    
    noOptUser = list(filter(lambda x: x not in haveRecordUser,user))
    
    optData = CountIp.to_dict()

    test_fea[attr] = test_fea.apply(lambda s:CountOpt(s['user'],optData,noOptUser),axis=1)
